In [1]:
import matplotlib.pyplot as plt
from lr_steps import *
import matplotlib.pyplot as plt
import pickle
import time

In [2]:
def run_lr_algo(d, r, b, sigma,
                lr_r, lr_f, beta_r, beta_f,
                m, avg_frac, stop_frac, T_r, T_f,
                times):
    x = np.random.randn(b, d, 1) * sigma
    regrets = np.zeros([times, m+1])
    for t in range(times):
        start_time = time.time()
        print(f"#Exp={t}")
        R, f, p, o, regrets_t = algorithm(x=x, r=r,
                                       lr_r=lr_r, lr_f=lr_f, beta1=beta_r, beta2=beta_f,
                                       m=m, T_r=T_r, T_f=T_f, avg_frac=avg_frac, stop_frac=stop_frac)
        regrets[t, :] = regrets_t
        print(regrets_t[-1])
        run_time_seconds = time.time() - start_time
        print(f"Running Time: {run_time_seconds // 60}:{run_time_seconds % 60}")

    return regrets

def plot_and_save(x_label, y_label, title, label_letter, x_axis, name):
    plt.grid()
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    for x_idx, x in enumerate(x_axis):
        mean = mean_regrets[x_idx]
        std = std_regrets[x_idx]
        plt.plot(x_axis, mean, label=label_letter+f"{x}")
        plt.fill_between(x_axis, mean-std, mean+std, alpha=0.5, color='r')

    plt.legend()
    plt.savefig(f"{name}.png")
    pickle.dump(mean_regrets, open(f'{name}_mean.p', 'wb'))
    pickle.dump(std_regrets, open(f'{name}_std.p', 'wb'))
    plt.clf()

In [3]:
b = 1000
d = 15
r = 3
sigma=1

lr_r = 1e-3
lr_f = 1e-1
beta_r = 0.9
beta_f = 0.9

m = 10
avg_frac=4
stop_frac=4
T_r=100
T_f=1_000

times = 10

In [5]:
regrets = run_lr_algo(d=d, r=r, b=b, sigma=sigma,
                      lr_r=lr_r, lr_f=lr_f, beta_r=beta_r, beta_f=beta_f,
                      m=m, avg_frac=avg_frac, stop_frac=stop_frac, T_r=T_r, T_f=T_f,
                      times=times)

plt.grid()
plt.title("logistic regression regret vs iteration")
plt.xlabel("#Iteration")
plt.ylabel("Regret")
plt.plot(range(m+1), regrets.T.mean(axis=1), label=r"$d$="+f"{d}")
plt.fill_between(range(m+1),
                 regrets.T.mean(axis=1)+regrets.T.std(axis=1), regrets.T.mean(axis=1)-regrets.T.std(axis=1),
                 color='r', alpha=0.5)
plt.legend()
name = f"log-reg_reg-vs-iter_d={d}_r={r}_b={b}"
plt.savefig(f"{name}.png")
pickle.dump(regrets, open(f'{name}.p', 'wb'))
plt.clf()

#Exp=0


 78%|███████▊  | 779/1000 [00:04<00:01, 191.96it/s]


KeyboardInterrupt: 

In [5]:
max_m = 10
m_step = 3

d_start = 5
d_max = 20
d_step = 3

m_list = list(range(1, max_m+1, m_step))
d_list = list(range(5, d_max+1, d_step))

mean_regrets = np.zeros([len(m_list), len(d_list)])
std_regrets = np.zeros([len(m_list), len(d_list)])
for m_idx, m in enumerate(m_list):
    for d_idx, d in enumerate(d_list):
        print(f"*** m={m}, d={d} ***")
        regrets = run_lr_algo(d=d, r=r, b=b, sigma=sigma,
                              lr_r=lr_r, lr_f=lr_f, beta_r=beta_r, beta_f=beta_f,
                              m=m, avg_frac=avg_frac, stop_frac=stop_frac, T_r=T_r, T_f=T_f,
                              times=times)
        mean_regrets[m_idx, d_idx] = regrets.min(axis=1).mean()
        std_regrets[m_idx, d_idx] = regrets.min(axis=1).std()

plot_and_save(x_label=r'$d$', y_label=f'Regret', title=r"logistic regression regret vs $d$"+f"r={r}",
              label_letter=r"$m$=", x_axis=d_list, name=f"reg-vs-d_r={r}_b={b}")

*** m=1, d=5 ***


TypeError: run_lr_algo() missing 1 required positional argument: 'sigma'

In [ ]:
max_m = 10
m_step = 3

s_start = 0.5
s_max = 3
s_step = 0.5

m_list = list(range(1, max_m+1, m_step))
s_list = np.linspace(s_start, s_max, int((s_max-s_start)/s_step)+1)

mean_regrets = np.zeros([len(m_list), len(d_list)])
std_regrets = np.zeros([len(m_list), len(d_list)])
for m_idx, m in enumerate(m_list):
    for s_idx, sigma in enumerate(s_list):
        print(f"*** m={m}, d={d} ***")
        regrets = run_lr_algo(d=d, r=r, b=b, sigma=sigma,
                              lr_r=lr_r, lr_f=lr_f, beta_r=beta_r, beta_f=beta_f,
                              m=m, avg_frac=avg_frac, stop_frac=stop_frac, T_r=T_r, T_f=T_f,
                              times=times)
        mean_regrets[m_idx, s_idx] = regrets.min(axis=1).mean()
        std_regrets[m_idx, s_idx] = regrets.min(axis=1).std()

plot_and_save(x_label=r'$\sigma_0$', y_label=f'Regret',
              title=r"logistic regression regret vs $\sigma_0$"+f"r={r}",
              label_letter=r"$m$=", x_axis=s_list, name=f"reg-vs-sigma_d={d}_b={b}")